In [40]:
import numpy as np
import pandas as pd

df = pd.read_csv("./2021Census_G01_VIC_SA1.csv")
total_pop = df["Tot_P_P"]
total_pop




0         435
1         184
2         377
3         584
4         358
         ... 
15477     493
15478       0
15479      21
15480      14
15481    9089
Name: Tot_P_P, Length: 15482, dtype: int64

In [41]:
# 1. Parents (Age 25 - 55) born in China 

df = pd.read_csv("./2021Census_G09F_VIC_SA1.csv")

china_born_df = df.loc[:, "P_China_25_44":"P_China_45_54"].copy()

SA1s = df.iloc[:, 0]

china_born_df.insert(0, "SA1 Code", SA1s)

china_born_df["Chinese Parents (25-55)"] = china_born_df["P_China_25_44"] + china_born_df["P_China_45_54"]

china_born_df = china_born_df.drop(china_born_df.columns[[1, 2]], axis=1)

china_born_df.sort_values("Chinese Parents (25-55)", ascending=False).head(20)

china_born_df.insert(2, "Total Pop", total_pop)
china_born_df["Prop_CN_Parents"] = china_born_df["Chinese Parents (25-55)"]/china_born_df["Total Pop"]
china_born_df.sort_values("Prop_CN_Parents", ascending=False)

china_born_df[china_born_df["SA1 Code"] == 20703116357]


,SA1 Code,Chinese Parents (25-55),Total Pop,Prop_CN_Parents
5003,20703116357,240,767,0.312907


In [42]:
# 2. Children with Parents born in China 
df = pd.read_csv("./2021Census_G08_VIC_SA1.csv")
chinese_parents_df = df[["SA1_CODE_2021", "Chinese_Tot_resp"]].copy()
chinese_parents_df.rename(columns={'Chinese_Tot_resp': 'Children w/ Chinese Parents'}, inplace=True)
chinese_parents_df.sort_values("Children w/ Chinese Parents", ascending=False)

chinese_parents_df.insert(2, "Total Pop", total_pop)
chinese_parents_df["Prop_Ch_w_CN_Parents"] = chinese_parents_df["Children w/ Chinese Parents"]/chinese_parents_df["Total Pop"]
chinese_parents_df.sort_values("Prop_Ch_w_CN_Parents", ascending=False)

chinese_parents_df[chinese_parents_df["SA1_CODE_2021"] == 20703116357]

,SA1_CODE_2021,Children w/ Chinese Parents,Total Pop,Prop_Ch_w_CN_Parents
5003,20703116357,547,767,0.713168


In [43]:
# 3. Children currently attending Primary School
# 4. Children currently attending Secondary School 
df = pd.read_csv("./2021Census_G15_VIC_SA1.csv")
students_df = df[["SA1_CODE_2021", "Primary_Tot_Primary_P", "Secondary_Tot_Secondary_P"]].copy()
students_df.rename(columns={"Primary_Tot_Primary_P": "Primary"}, inplace=True)
students_df.rename(columns={"Secondary_Tot_Secondary_P": "Secondary"}, inplace=True)
students_df.sort_values("Secondary", ascending=False)

students_df.insert(3, "Total Pop", total_pop)
students_df["Prop_Primary"] = students_df["Primary"]/students_df["Total Pop"]
students_df["Prop_Secondary"] = students_df["Secondary"]/students_df["Total Pop"]

# students_df.sort_values("Prop_Primary", ascending=False)
students_df.sort_values("Prop_Secondary", ascending=False)

# students_df

,SA1_CODE_2021,Primary,Secondary,Total Pop,Prop_Primary,Prop_Secondary
8104,21005125019,0,3,4,0.000000,0.750000
12410,21303135111,0,4,6,0.000000,0.666667
8390,21101125407,0,4,6,0.000000,0.666667
1247,20302148603,20,347,543,0.036832,0.639042
142,20101100825,22,179,367,0.059946,0.487738
...,...,...,...,...,...,...
15365,21704147808,0,0,0,NaN,NaN
15387,21704147902,0,0,0,NaN,NaN
15436,21704147954,0,0,0,NaN,NaN
15455,21704148013,0,0,0,NaN,NaN


In [44]:
# 5. Families with total reported income of >$100,000 
Sheet_A = pd.read_csv("./2021Census_G58A_VIC_SA1.csv")
Sheet_B = pd.read_csv("./2021Census_G58B_VIC_SA1.csv")
income_df = Sheet_A[["SA1_CODE_2021", "I_1_149_Tot", "I_150_299_Tot", "I_300_399_Tot", "I_400_499_Tot", "I_500_649_Tot", "I_650_799_Tot", "I_800_999_Tot", "I_1000_1249_Tot", "I_1250_1499_Tot", "I_1500_1749_Tot", "I_1750_1999_Tot", "I_2000_2499_Tot", "I_2500_2999_Tot", "I_3000_3499_Tot", "I_3500_3999_Tot"]].copy()

high_income = Sheet_B[["I_4000more_Tot"]].copy()
income_df = income_df.merge(high_income, left_index=True, right_index=True)
income_df["High Income"] = income_df[["I_2000_2499_Tot", "I_2500_2999_Tot", "I_3000_3499_Tot", "I_3500_3999_Tot","I_4000more_Tot"]].sum(axis=1)
income_df["Total"] = income_df[["I_1_149_Tot", "I_150_299_Tot", "I_300_399_Tot", "I_400_499_Tot", "I_500_649_Tot", "I_650_799_Tot", "I_800_999_Tot", "I_1000_1249_Tot", "I_1250_1499_Tot", "I_1500_1749_Tot", "I_1750_1999_Tot", "I_2000_2499_Tot", "I_2500_2999_Tot", "I_3000_3499_Tot", "I_3500_3999_Tot", "I_4000more_Tot"]].sum(axis=1)
income_df["Prop_Aff"] = income_df["High Income"] / income_df["Total"]
income_df = income_df[["SA1_CODE_2021", "High Income", "Total", "Prop_Aff"]]


income_df





 

,SA1_CODE_2021,High Income,Total,Prop_Aff
0,20101100101,30,39,0.769231
1,20101100102,11,11,1.000000
2,20101100105,8,14,0.571429
3,20101100106,37,57,0.649123
4,20101100107,3,3,1.000000
...,...,...,...,...
15477,21704148035,43,55,0.781818
15478,29797979991,0,0,NaN
15479,29797979992,0,0,NaN
15480,29797979993,0,0,NaN


In [49]:
# Concatenate all the DataFrames in the `dfs` list into a single DataFrame
combined_df = pd.concat([china_born_df, chinese_parents_df.drop(["SA1_CODE_2021", "Total Pop"], axis=1), students_df.drop(["SA1_CODE_2021", "Total Pop"], axis=1), income_df.drop("SA1_CODE_2021", axis=1)], axis=1)

# Drop SA1s with <50 people because not efficient + high chance of inaccurate data
combined_df = combined_df.drop(combined_df[combined_df['Total Pop'] < 50].index)

# first normalize the proportions to be on the same scale
combined_df['Prop_CN_Parents_z'] = combined_df['Prop_CN_Parents'] / combined_df['Prop_CN_Parents'].max()
combined_df['Prop_Ch_w_CN_Parents_z'] = combined_df['Prop_Ch_w_CN_Parents'] / combined_df['Prop_Ch_w_CN_Parents'].max()
combined_df['Prop_Secondary_z'] = combined_df['Prop_Secondary'] / combined_df['Prop_Secondary'].max()
combined_df['Prop_Primary_z'] = combined_df['Prop_Primary'] / combined_df['Prop_Primary'].max()
combined_df['Prop_Aff_z'] = combined_df['Prop_Aff'] / combined_df['Prop_Aff'].max()

# create a score with weights
combined_df['Score'] = (combined_df['Prop_CN_Parents_z'] * 0.55 +
               combined_df['Prop_Ch_w_CN_Parents_z'] * 0.05 +
               combined_df['Prop_Secondary_z'] * 0.35 +
               combined_df['Prop_Primary_z'] * 0.1 +
               combined_df['Prop_Aff_z'] * 0.05)

# # made a list of all valid SA3 regions
sa3_region_df = pd.read_csv("./SA3 Region - Sheet1.csv")
valid_SA3 = sa3_region_df['SA3s within 20km Radius'].tolist()

# filtered combined_df to only keep rows which have SA3 values in the valid SA3 list
valid_regions = combined_df[combined_df['SA1 Code'].astype(str).str.startswith(tuple(str(num) for num in valid_SA3))]
valid_regions

# sort by score and select top 20
top_regions = valid_regions.sort_values('Score', ascending=False).head(10)
top_regions


,SA1 Code,Chinese Parents (25-55),Total Pop,Prop_CN_Parents,Children w/ Chinese Parents,Prop_Ch_w_CN_Parents,Primary,Secondary,Prop_Primary,Prop_Secondary,High Income,Total,Prop_Aff,Prop_CN_Parents_z,Prop_Ch_w_CN_Parents_z,Prop_Secondary_z,Prop_Primary_z,Prop_Aff_z,Score
5003,20703116357,240,767,0.312907,547,0.713168,9,22,0.011734,0.028683,10,18,0.555556,0.900563,0.852991,0.044885,0.050020,0.555556,0.586448
5004,20703116358,163,558,0.292115,359,0.643369,9,5,0.016129,0.008961,6,13,0.461538,0.840720,0.769507,0.014022,0.068754,0.461538,0.535732
4984,20703116334,248,909,0.272827,537,0.590759,18,33,0.019802,0.036304,30,58,0.517241,0.785210,0.706582,0.056809,0.084411,0.517241,0.521381
4729,20702115755,92,408,0.225490,224,0.549020,25,30,0.061275,0.073529,17,24,0.708333,0.648972,0.656659,0.115062,0.261199,0.708333,0.491576
4998,20703116352,79,321,0.246106,210,0.654206,9,11,0.028037,0.034268,8,15,0.533333,0.708305,0.782468,0.053624,0.119517,0.533333,0.486078
4956,20703116303,101,466,0.216738,321,0.688841,14,30,0.030043,0.064378,24,24,1.000000,0.623783,0.823894,0.100741,0.128066,1.000000,0.482341
5503,20802117757,125,547,0.228519,306,0.559415,9,8,0.016453,0.014625,9,9,1.000000,0.657689,0.669093,0.022886,0.070137,1.000000,0.460208
4965,20703116312,178,777,0.229086,487,0.626770,15,19,0.019305,0.024453,15,27,0.555556,0.659321,0.749653,0.038265,0.082293,0.555556,0.449509
4997,20703116351,180,762,0.236220,488,0.640420,15,25,0.019685,0.032808,5,30,0.166667,0.679854,0.765979,0.051340,0.083913,0.166667,0.446912
11242,21205132119,59,340,0.173529,163,0.479412,35,32,0.102941,0.094118,33,40,0.825000,0.499426,0.573404,0.147279,0.438815,0.825000,0.440034


In [46]:
[top_regions["Total"].sum(), top_regions["Total Pop"].sum()]

[258, 5855]